In [1]:
from itertools import product

from itertools import islice

def batched(iterable, n):
    if n < 1:
        raise ValueError('n must be at least one')
    it = iter(iterable)
    while batch := tuple(islice(it, n)):
        yield batch

def get_batch(iterable, n, id):
    if n < 1:
        raise ValueError('n must be at least one')
    i = n * id
    return tuple(islice(iter(iterable), i, i + n))

def perm_cycles(a, b):
    todo = set(a)

    while len(todo) > 0:
        do = min(todo)

        cycle = [do]
        while do in todo:
            todo.remove(do)
            do_i = next( i for i, val in enumerate(a) if val == do )
            don = b[do_i]
            if don == do or don == cycle[0]: break

            cycle.append(don)
            do = don
        if len(cycle) > 1:
            yield tuple(cycle)

In [2]:
def improve_facet_sums(ar):
    facet_sums = tuple(map(sum, batched(ar, 4)))

    min_facet_id, min_facet_sum = min(enumerate(facet_sums), key=lambda iv: iv[1])
    max_facet_id, max_facet_sum = max(enumerate(facet_sums), key=lambda iv: iv[1])

    facet_diff = max_facet_sum - min_facet_sum
    if facet_diff <= 0: return None

    min_facet = get_batch(ar, 4, min_facet_id)
    max_facet = get_batch(ar, 4, max_facet_id)

    try:
        best_swap = max((
            choice
            for choice in product(enumerate(min_facet), enumerate(max_facet))
            if choice[1][1] - choice[0][1] <= facet_diff
        ),
            key=lambda choice: choice[1][1] - choice[0][1]
        )
    except ValueError:
        return None

    return 4*min_facet_id + best_swap[0][0], 4*max_facet_id + best_swap[1][0]

In [3]:
ar = list(range(1, 25))

# TODO: longer cycle detection
last = None
for _ in range(100):
    print(ar)
    move = improve_facet_sums(ar)
    if move is None: break

    i, j = move
    if (i,j) == last or (j,i) == last: break

    print(f'( {i} {j} )')
    ar[i], ar[j] = ar[j], ar[i]

    last = i,j

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
( 0 23 )
[24, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 1]
( 4 19 )
[24, 2, 3, 4, 20, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 5, 21, 22, 23, 1]
( 1 22 )
[24, 23, 3, 4, 20, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 5, 21, 22, 2, 1]
( 5 18 )
[24, 23, 3, 4, 20, 19, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 6, 5, 21, 22, 2, 1]
( 8 15 )
[24, 23, 3, 4, 20, 19, 7, 8, 16, 10, 11, 12, 13, 14, 15, 9, 17, 18, 6, 5, 21, 22, 2, 1]
( 16 0 )
[17, 23, 3, 4, 20, 19, 7, 8, 16, 10, 11, 12, 13, 14, 15, 9, 24, 18, 6, 5, 21, 22, 2, 1]
( 23 7 )
[17, 23, 3, 4, 20, 19, 7, 1, 16, 10, 11, 12, 13, 14, 15, 9, 24, 18, 6, 5, 21, 22, 2, 8]
( 2 18 )
[17, 23, 6, 4, 20, 19, 7, 1, 16, 10, 11, 12, 13, 14, 15, 9, 24, 18, 3, 5, 21, 22, 2, 8]
( 5 21 )
[17, 23, 6, 4, 20, 22, 7, 1, 16, 10, 11, 12, 13, 14, 15, 9, 24, 18, 3, 5, 21, 19, 2, 8]
( 10 12 )
[17, 23, 6, 4, 20, 22, 7, 1, 

In [4]:
tuple(map(sum, batched(ar, 4)))

(50, 50, 51, 49, 50, 50)

In [21]:
a = tuple(range(1, 25))
b = tuple(ar)

print(a)
print(b)
print(' '.join(f'({" ".join(map(str, c))})' for c in perm_cycles(a, b)))

(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24)
(17, 23, 6, 4, 20, 22, 7, 1, 16, 10, 13, 12, 11, 14, 15, 9, 24, 18, 3, 5, 21, 19, 2, 8)
(1 17 24 8) (2 23) (3 6 22 19) (5 20) (9 16) (11 13)
